In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.utils import Sequence
from keras.layers import Conv2D, MaxPooling2D
from qkeras import *

from keras.utils import Sequence
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping

import os
import random

pi = 3.14159265359

maxval=1e9
minval=1e-9

2024-02-12 22:31:32.616880: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from dataprep import *
from loss import *
from models import *

In [3]:
batch_size = 5000
val_batch_size = 5000
train_file_size = 142
val_file_size = 6

training_generator = CustomDataGenerator(
    data_directory_path = "/uscms/home/lagray/nobackup/smart-pixels/dataset8/unflipped-positive/data/",
    labels_directory_path = "/uscms/home/lagray/nobackup/smart-pixels/dataset8/unflipped-positive/labels/",
    is_directory_recursive = False,
    file_type = "parquet",
    data_format = "3D",
    batch_size = batch_size,
    file_count = train_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    input_shape = (20,13,21),
    transpose = (0,2,3,1),
)

validation_generator = CustomDataGenerator(
    data_directory_path = "/uscms/home/lagray/nobackup/smart-pixels/dataset8/unflipped-positive/data/",
    labels_directory_path = "/uscms/home/lagray/nobackup/smart-pixels/dataset8/unflipped-positive/labels/",
    is_directory_recursive = False,
    file_type = "parquet",
    data_format = "3D",
    batch_size = val_batch_size,
    file_count = val_file_size,
    to_standardize= True,
    include_y_local= False,
    labels_list = ['x-midplane','y-midplane','cotAlpha','cotBeta'],
    input_shape = (20,13,21),
    transpose = (0,2,3,1),
    files_from_end=True,
)

In [4]:
# compiles model
# loss = custom_loss
model=CreateModel((13,21,20),n_filters=5,pool_size=4)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13, 21, 20)]      0         
                                                                 
 q_separable_conv2d (QSepara  (None, 11, 19, 5)        285       
 bleConv2D)                                                      
                                                                 
 q_activation (QActivation)  (None, 11, 19, 5)         0         
                                                                 
 q_separable_conv2d_1 (QSepa  (None, 9, 17, 5)         75        
 rableConv2D)                                                    
                                                                 
 q_activation_1 (QActivation  (None, 9, 17, 5)         0         
 )                                                               
                                                             

In [5]:
model.compile(optimizer=Adam(learning_rate=0.001), loss=custom_loss)

In [6]:
# training
es = EarlyStopping(
    patience=50,
    restore_best_weights=True
)

fingerprint = '%08x' % random.randrange(16**8)

base_dir = f'./weights-{fingerprint}-checkpoints'
os.mkdir(base_dir)
checkpoint_filepath = base_dir + '/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'
mcp = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=False,
)

class ScalePrintingCallback(keras.callbacks.Callback):    
    def on_epoch_end(self, epoch, logs=None):
        scale_layer = self.model.layers[-1]
        print(
            f"scaling layer ({epoch}):", 
            scale_layer.scale, 
            tf.math.softplus(scale_layer.scale)
        )

print_scale = ScalePrintingCallback()

history = model.fit(x=training_generator,
                    validation_data=validation_generator,
                    callbacks=[mcp],
                    epochs=1000,
                    shuffle=False, # shuffling now occurs within the data-loader
                    verbose=1)

2024-02-12 22:41:29.264292: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 104.14MiB (rounded to 109200128)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-02-12 22:41:29.264436: W tensorflow/core/common_runtime/bfc_allocator.cc:491] *___________________________________________________________________________________________________


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.